In [1]:
#%reset
import tensorflow as tf
import os,sys
import glob
from IPython.display import Image
sys.path.append('/home/dagutman/devel/tensorflow-vgg/')  # Change to your local path

# Downloaded the model from the link on here https://github.com/machrisaa/tensorflow-vgg
## URL was https://mega.nz/#!xZ8glS6J!MAnE91ND_WyfZ_8mvkuSa2YcA7q-1ehfSm-Q1fxOvvs

import vgg19_trainable as vgg19
# and placed file in the directory I am running jupyter from directory
from scipy.misc import imread
import numpy as np

In [2]:
n_classes = 2
LEARN_RATE = 1e-6
n_epochs = 25

In [4]:
tf.reset_default_graph()

model = vgg19.Vgg19('./vgg19.npy')

# Placehol\ders and feed data
with tf.name_scope("graph_inputs"):
    model.images = tf.placeholder("float", shape=[None, 224, 224, 3])
    model.labels = tf.placeholder("float", shape=[None, n_classes])

# Actual model
with tf.name_scope("vgg19_content"):
    model.build(model.images)

In [5]:
"""This module provides the a softmax cross entropy loss for training FCN.
In order to train VGG first build the model and then feed apply vgg_fcn.up
to the loss. The loss function can be used in combination with any optimizer
(e.g. Adam) to finetune the whole model.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

def loss(logits, labels, num_classes, head=None):
    """Calculate the loss from the logits and the labels.
    Args:
      logits: tensor, float - [batch_size, num_classes].
          Use vgg_fcn.upscore as logits.
      labels: Labels tensor, int32 - [batch_size, num_classes].
          The ground truth of your data.
      head: numpy array - [num_classes]
          Weighting the loss of each class
          Optional: Prioritize some classes
    Returns:
      loss: Loss tensor of type float.
    """
    with tf.name_scope('loss'):
        
        logits = tf.reshape(logits, (-1, num_classes))
        labels = tf.to_float(tf.reshape(labels, (-1, num_classes)))
        
        epsilon = tf.constant(value=1e-4)
        logits = tf.nn.softmax(logits) + epsilon

        if head is not None:
            cross_entropy = -tf.reduce_sum(tf.multiply(labels * tf.log(logits),
                                           head), reduction_indices=[1])
        else:
            cross_entropy = -tf.reduce_sum(
                labels * tf.log(logits), reduction_indices=[1])

        cross_entropy_mean = tf.reduce_mean(cross_entropy,
                                            name='xentropy_mean')
        tf.add_to_collection('losses', cross_entropy_mean)

        loss = tf.add_n(tf.get_collection('losses'), name='total_loss')
    return loss

In [6]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

[u'graph_inputs/Placeholder',
 u'graph_inputs/Placeholder_1',
 u'vgg19_content/mul/y',
 u'vgg19_content/mul',
 u'vgg19_content/Const',
 u'vgg19_content/split/split_dim',
 u'vgg19_content/split',
 u'vgg19_content/sub/y',
 u'vgg19_content/sub',
 u'vgg19_content/sub_1/y',
 u'vgg19_content/sub_1',
 u'vgg19_content/sub_2/y',
 u'vgg19_content/sub_2',
 u'vgg19_content/concat/axis',
 u'vgg19_content/concat',
 u'vgg19_content/conv1_1/truncated_normal/shape',
 u'vgg19_content/conv1_1/truncated_normal/mean',
 u'vgg19_content/conv1_1/truncated_normal/stddev',
 u'vgg19_content/conv1_1/truncated_normal/TruncatedNormal',
 u'vgg19_content/conv1_1/truncated_normal/mul',
 u'vgg19_content/conv1_1/truncated_normal',
 u'vgg19_content/conv1_1/conv1_1_filters/initial_value',
 u'vgg19_content/conv1_1/conv1_1_filters',
 u'vgg19_content/conv1_1/conv1_1_filters/Assign',
 u'vgg19_content/conv1_1/conv1_1_filters/read',
 u'vgg19_content/conv1_1/truncated_normal_1/shape',
 u'vgg19_content/conv1_1/truncated_normal_1/

In [7]:
model.cost = loss(model.prob, model.labels, num_classes=n_classes)

In [8]:
model.optimizer = tf.train.AdamOptimizer(LEARN_RATE).minimize(model.cost)

In [9]:
brcaThumbs_Train_Path = '/home/dagutman/devel/DSA_CBIR_Prototypes/dataSetGeneration/localCache/Exp1_BRCA_vs_GBM/TRAINING/BRCA_Thumbs/*.jpg'
gbmThumbs_Train_Path = '/home/dagutman/devel/DSA_CBIR_Prototypes/dataSetGeneration/localCache/Exp1_BRCA_vs_GBM/TRAINING/GBM_Thumbnails/*.jpg'

In [10]:
brcaThumbs_Train = glob.glob(brcaThumbs_Train_Path)
gbmThumbs_Train = glob.glob(gbmThumbs_Train_Path)

In [11]:
modelImageData = []

#gbm is class 0
for x in gbmThumbs_Train:
    modelImageData.append( (x,0) )                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
    
#brca is class 1
for x in brcaThumbs_Train:
    modelImageData.append( (x,1) )

In [12]:
print(modelImageData[2])

('/home/dagutman/devel/DSA_CBIR_Prototypes/dataSetGeneration/localCache/Exp1_BRCA_vs_GBM/TRAINING/GBM_Thumbnails/TCGA-06-0875-01Z-00-DX1.c04e2f8e-868d-455b-88b0-af5ee89bf319.svs.macrothumb.jpg', 0)


In [13]:
# Initialize
batch_size = 1
images = np.zeros([batch_size, 224, 224, 3])
labels = np.zeros([batch_size, n_classes])

In [14]:
print(images.shape)
print(labels[0, :])

(1, 224, 224, 3)
[ 0.  0.]


In [16]:
from random import shuffle

shuffle(modelImageData)

class LinePrinter():
    """
    Print things to stdout on one line dynamically
    """
    def __init__(self,data):
        sys.stdout.write("\r\x1b[K"+data.__str__())
        sys.stdout.flush()
        
#         output = "Total Processed: %d: Processing %d out of %d; File %s" % (totalProcessed, curBatchProcessed, batchSize, img['name'])
#             LinePrinter(output)
        
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    print("epoch\timidx\tcost")
    print("-------------------")
    
    costs = []
    
    for epoch in range(n_epochs):
        
        # load image and label
        for imidx, _ in enumerate(modelImageData):
            ## This needs to be modified to load in batches of batch_size
            #print(imidx)
            #continue
            
            # read image
            images[0, :, :, :] = imread(modelImageData[imidx][0])
            
            # make a "one-hot" label 
            labels[0, modelImageData[imidx][1]] = 1
        
            feed = {model.images: images,
                   model.labels: labels}

            _, cost = sess.run([model.optimizer, model.cost], feed_dict=feed)
            
            costs.append(cost)

            output = "{}\t{}\t{}".format(epoch, imidx, cost)
            LinePrinter(output)

epoch	imidx	cost
-------------------
24	499	1.38637459278

In [ ]:
import matplotlib.pyplot as plt
plt.plot(costs[2:])
plt.ylabel('costs')
plt.show()

imnageData

In [ ]:
tf.reset_default_graph()